<img src="images/Picture0.png" width=200x />

# Notebook 06 - Infeasible Systems

### Covered in this notebook:
* Infeasible systems
* Irreducible Infeasible Systems (IIS)
* Using Gurobi to relax constraints

### Prerequisites:
- N02, N03, N04

### Credits:
* https://support.gurobi.com/hc/en-us/articles/360029969391-How-do-I-determine-why-my-model-is-infeasible
* Gurobipy documentation
* Gurobipy functional code examples
* COB 291: Introduction to Management Science Teaching Supplement, William Christian, Spring 2004

In [1]:
import gurobipy as gp
from gurobipy import GRB

## Interstitial: Reading and Writing Models

Before we discuss infeasibility, let's take a moment to briefly discuss exporting and importing models from files.  In general, writing a model to a filetype is done by calling the `Model.write()` method on the model, as in the following example:

```
# model.write("filename.ext")
```

Meanwhile, reading a model in from a file is done by using the Gurobipy function `read()`:

```
# model = gp.read("filename.ext")
```

Gurobi uses two/four main storage types for a model and one main storage type for a solution.  The <strong>MPS</strong> filetype stores a model in full detail, while the <strong>LP</strong> filetype is designed to be human-readable (but stores coefficients with fewer decimal digits, and does not preserve the order of variables).  The REW and RLP file formats are equivalent to the MPS and LP formats, respectively, but do not use the user-defined names for elements, instead creating names based on the element type and the order in which it was introduced.

Solutions are stored in the <strong>SOL</strong> format.  They can be read in or out, sparing one the step of running the model's optimization if it has been done in the past.

A few more of the 15+ filetypes which Gurobi can use include the following:

* DLP and DUA to store the dual formulation of a model
* JSON to store solutions in a highly thorough, non-proprietary, machine-parseable format
* MST to store a known feasible solution for the solver to start with before optimizing
* HNT to store hints toward feasible solutions

Try exporting your model and solutions from N03 to get a feeling for how the different file formats look.

---

## Infeasible Systems

A system of equations which is impossible to solve is known as <strong>infeasible</strong> or <strong>overconstrained</strong>, and occurs when the constraints of a system compete with one another such that there does not exist a region of overlap.

Infeasibility must be addressed by the modeler, and cannot be resolved by the machine on its own.  After all, if a constraint which requires that product be affordably priced conflicts with one that prefers elite suppliers, how would the machine be able to say which constraint to drop?  Resolving infeasibility falls within the realm of the human design.

Nevertheless, the tool can grant us heavy insight.  Resolving an infeasible system requires either removing or relaxing one or more contraints.  Gurobi has tools to both help us identify conflicting restraints and to relax constraints gradually while preserving the model's overall shape.

### Computing an Irreducible Infeasible System (IIS)

Let's first look at how to determine conflicting constraints.  If after optimizing a model, a solution is not delivered, we will know that our system is infeasible.  The model's `Status` attribute will inform us of whether the model is explicitly infeasible (`INFEASIBLE`) or whether the model could be either infeasible or unbounded (`INF_OR_UNBD`).

Gurobi can compute an <strong>Irreducible Infeasible System</strong> from our model, which is a subset of constraints and variable bounds defined such that:

* the set is infeasible
* if <i>any</i> constraint or bound in the set were removed, the set would be feasible

Then we need only consider which one constraint or bound in the system to remove, adjust, or relax to resolve the conflict.  Note that a system may contain more than one IIS, and the one which Gurobi returns may not be the one with the lowest cardinality.

We create an IIS by calling the `Model.computeIIS()` method on the model after attempting to optimize it.  We can then write the IIS to an ILP file to examine it.

## Exercise: Overconstrained Production (Part One)

Read in the example model in `overconstrained.mps`.  The situation is as below:

<img src="images/N06_images/handyPlan.png"/>

Optimize the model, create an IIS, and write it to a file.  Use the IIS to make manual adjustments to your system such that it is no longer overconstrained.  Evaluate your own performance: have you retained the spirit of the model?  Would this be reasonable to execute?

<i>Note: Use the `Model.remove()`, `Model.getConstrByName()`, and `Model.getVarByName()` methods to help rework your model, as well as reassigning attributes in place.</i>

In [ ]:
# solve here:


### Using Gurobi to relax constraints

As an alternative to manually relaxing constraints, we can also guide Gurobi into doing so for us.  The method `Model.feasRelaxS()` (the "s" standing for "simplified"; `Model.feasRelax()` is also available) iteratively adjusts the constraints of a model until violations no longer occur.  Note that this method is destructive; `feasRelaxS` will rewrite your model, so if you would like to keep your original as-written, be sure to copy, export, or otherwise save it before running `feasRelaxS`.

`feasRelaxS` runs by minimizing the <strong>cost</strong> of the relaxation while guaranteeing that a solution exists.  The cost represents how far the new model is from the original, and can be defined in one of three ways:
* <strong>Cost function 0</strong>: the sum of the magnitudes of the bound and constraint violations
* <strong>Cost function 1</strong>: the sum of the squares of the magnitudes of the bound and constraint violations
* <strong>Cost function 2</strong>: the total number of bound and constraint violations

It is up to you which you choose, but note that for smaller models, you can likely execute Cost Function 2 well enough on your own by examining your IIS.

In all, `feasRelaxS()`takes the following arguments:
* `relaxobjtype`: an integer (0, 1, or 2) indicating the <strong>cost function</strong> used to find the minimum cost relaxation
* `minrelax`: Boolean indicating the type of feasibility relaxation to perform.
    * If `False`, `feasRelaxS` runs according to the previously-defined cost function.
    * If `True`, it optimizes both according to the cost function <i>and</i> so that the original objective is optimized as well.  This can be computationally expensive.
* `vrelax`: Boolean indicating whether variable bounds can be relaxed.
* `crelax`: Boolean indicating whether constraints can be relaxed.

## Exercise: Overconstrained Production (Part Two)

Use `feasRelaxS()` on the infeasible model defined earlier.  Choose cost function, etc. as seems reasonable to you.  Once the relaxation is performed, compare the result to that of your revised model.  Tweak your parameters if the result seems impractical.  How do the models differ?  Which solution appears more practically useful?  What are the merits of each method?

In [ ]:
# solve here: 
